In [1]:
%%time
from rdflib import Graph, plugin, URIRef, Literal, BNode, RDF
from rdflib.serializer import Serializer
from rdflib.namespace import SKOS
import rdflib
import re
import sys
import os
import json
import utils
import lmdb
import pickle
import numpy as np
from io import BytesIO
from datetime import datetime

# keep changing, cantact with Alireza
sys.path.append("/home/bobby/aida_copy/AIDA/AIDA-Interchange-Format/python/")
import aida_interchange.aifutils as aifutils
from aida_interchange.Bounding_Box import Bounding_Box


CPU times: user 4.1 s, sys: 1.01 s, total: 5.11 s
Wall time: 24.8 s


In [2]:
 '''
import shutil,errno
src = '/dvmm-filer2/projects/AIDA/brian/...'
dst = '/home/bobby/aida_copy/AIDA/M18_copy/data/cu_ttl/_ttl_06/'
# def copyanything(src, dst):
#     try:
#         shutil.copytree(src, dst)
#     except OSError as exc: # python >2.5
#         if exc.errno == errno.ENOTDIR:
#             shutil.copy(src, dst)
#         else: raise
shutil.rmtree(dst)
shutil.copytree(src, dst)
# shutil.copyfile(src, dst)


# unzip
import zipfile
for zip_file in os.listdir(dst):
    zip_file = os.path.join(dst,zip_file)
    print(zip_file)
    zip_ref = zipfile.ZipFile(zip_file, 'r')
    zip_ref.extractall(dst)
    zip_ref.close()
# renaming 
old_name, new_name = 'dry3', 'm18_dryrun_E2'
os.rename(os.path.join(dst, old_name), os.path.join(dst, new_name))
print(old_name, '->', new_name)
old_name, new_name = 'dry3_i_c', 'm18_dryrun_im_E2'
os.rename(os.path.join(dst, old_name), os.path.join(dst, new_name))
print(old_name, '->', new_name)
#'''

"\nimport shutil,errno\nsrc = '/dvmm-filer2/projects/AIDA/brian/...'\ndst = '/home/bobby/aida_copy/AIDA/M18_copy/data/cu_ttl/_ttl_06/'\n# def copyanything(src, dst):\n#     try:\n#         shutil.copytree(src, dst)\n#     except OSError as exc: # python >2.5\n#         if exc.errno == errno.ENOTDIR:\n#             shutil.copy(src, dst)\n#         else: raise\nshutil.rmtree(dst)\nshutil.copytree(src, dst)\n# shutil.copyfile(src, dst)\n\n\n# unzip\nimport zipfile\nfor zip_file in os.listdir(dst):\n   zip_file = os.path.join(dst,zip_file)\n   print(zip_file)\n   zip_ref = zipfile.ZipFile(zip_file, 'r')\n   zip_ref.extractall(dst)\n   zip_ref.close()\n# renaming \nold_name, new_name = 'dry3', 'm18_dryrun_E2'\nos.rename(os.path.join(dst, old_name), os.path.join(dst, new_name))\nprint(old_name, '->', new_name)\nold_name, new_name = 'dry3_i_c', 'm18_dryrun_im_E2'\nos.rename(os.path.join(dst, old_name), os.path.join(dst, new_name))\nprint(old_name, '->', new_name)\n#"

In [3]:
def top_dict(kv_dict, num = 2):
    k_list = list(kv_dict.keys())[:num]
    subdict =  {k: kv_dict[k] for k in k_list}
    print('\ndict len:', len(kv_dict))
    return subdict

def check_usc_grounding_dict(usc_grounding_dict, child_dict):
    # test
#     key = 'http://www.isi.edu/gaia/entities/b7ec53ef-3021-4eda-9429-7c87ccd1dc0d'
#     if 'usc_vision' in usc_grounding_dict[key]['grounding'].values():
#         print('find it')
#     print('arrrr', usc_grounding_dict[key]['grounding']['system'])
    
    # Check Point: grounding_dict printing
    top,i = 3,0
    print('usc_grounded')
    for en in usc_grounding_dict.keys():
        for img_id in usc_grounding_dict[en]['grounding']:
            if img_id  =='system':
                continue
            n_b = len(usc_grounding_dict[en]['grounding'][img_id]['bbox'])
            if n_b>0:
                img_dict = usc_grounding_dict[en]['grounding'][img_id]
                if i < top: 
                    i+=1
                    print(en)
                    print(child_dict[img_id], img_id, usc_grounding_dict[en]['grounding'][img_id])
                    print(sum(sum(img_dict['grounding_features'])),sum(sum(img_dict['instance_features'])))
                    print('\n')

# cu_grndg_ent_pref = 'http://www.columbia.edu/AIDA/DVMM/Entities/GroundingBox/'+merge_version+'/'
# cu_grndg_type_pref = 'http://www.columbia.edu/AIDA/DVMM/TypeAssertions/GroundingBox/'+merge_version+'/'
# cu_grndg_clstr_img_pref = 'http://www.columbia.edu/AIDA/DVMM/Clusters/BoxOverlap/'+merge_version+'/'
# cu_grndg_clstr_txt_pref = 'http://www.columbia.edu/AIDA/DVMM/Clusters/Grounding/'+merge_version+'/'
        
    
def merge_usc(usc_dict_path, grounding_dict, child_dict):    
    grounded = dict((k,v) for k,v in grounding_dict.items() if len(v['grounding']) > 0)
    # print('grounded', top_dict(grounded),'\n')
    usc_grounding_dict = pickle.load(open(usc_dict_path,'rb'))
    check_usc_grounding_dict(usc_grounding_dict, child_dict)
    
    print('intersection num', len(list(set(usc_grounding_dict.keys()).intersection(set(grounded.keys())))))
    print('difference (num added from usc)', len(list(set(usc_grounding_dict.keys()).difference(set(grounded.keys())))))
    conflict_set = list(set(usc_grounding_dict.keys()).intersection(set(grounded.keys())))
    print('conflict entities:',conflict_set)
    for k in conflict_set:
        del usc_grounding_dict[k]

    grounding_dict.update(usc_grounding_dict) # if key is same, the new pair will be appended  
#     print('usc_grounding_dict', top_dict(usc_grounding_dict),'\n')
#     print('merged_grounding_dict', top_dict(merged_grounding_dict),'\n')
    print('usc_grounded_entity in merged_grounding_dict',grounding_dict[list(usc_grounding_dict.keys())[0]])
    return grounding_dict

In [4]:
%%time
# it should returns 
print(datetime.now())
working_path = '/home/bobby/aida_copy/AIDA/M18_copy/data/'
corpus_path = '/dvmm-filer2/projects/AIDA/data/ldc_eval_m18/LDC2019E42_AIDA_Phase_1_Evaluation_Source_Data_V1.0/'
# corpus_path = '/dvmm-filer2/projects/AIDA/data/ldc_isi_dryrun3/dryrun-updated_tmp/dryrun/'
#corpus_path = '/dvmm-filer2/projects/AIDA/data/ldc_isi_dryrun3/dryrun/' # D3
#corpus_path = '/dvmm-filer2/projects/AIDA/data/ldc_isi_dryrun/dryrun/' # D2
img_path = corpus_path
parent_child_tab = corpus_path + 'docs/parent_children.sorted.tab' # comes from others
print('Check Point: ISI raw data change',corpus_path)

p_f = 'PT002_r2' # E5
p_f_share = 'E2'

#grounding path
grounding_dict_path = working_path + 'all_features/grounding_dict_'+p_f+'.pickle'

parent_dict, child_dict = utils.create_dict(parent_child_tab)
grounding_dict = pickle.load(open(grounding_dict_path,'rb'))
print('Check Point: version change',grounding_dict_path)
print('grounding_dict', top_dict(grounding_dict),'\n')


# Todo: change folder name
rpi_ttl_path = working_path + 'rpi_ttl/RPI_TA1_'+p_f # 17th May
# 'RPI_TA1_20190504' 'rpi_ttl/RPI_dryrun_'+p_f
print('Check Point: RPI path change',rpi_ttl_path)

# Todo: change path, version; two ttl folders from Brian
cuttl_path = '/dvmm-filer2/projects/AIDA/brian/'
update_version = 'm18/' #'dry_ttl_0625/' #'dry_ttl_0519/' # dry_ttl_0502/
cu_ttl_path = cuttl_path + update_version + 'm18_'+p_f  #working_path + 'cu_ttl/'+update_version+'m18_'+p_f
cu_ttl_ins_path = cuttl_path + update_version + 'm18_i_c_'+p_f_share #working_path + 'cu_ttl/'+update_version+'m18_i_c_'+p_f_share
print('Check Point: Brian cu_ttl path change',cu_ttl_path,cu_ttl_ins_path)


rpi_ttl_list = set([f.split('.')[0] for f in os.listdir(rpi_ttl_path)])
cu_ttl_list = set([f.split('.')[0] for f in os.listdir(cu_ttl_path)])
cu_ttl_ins_list = set([f.split('.')[0] for f in os.listdir(cu_ttl_ins_path)])

cu_pref = 'http://www.columbia.edu/AIDA/DVMM/'
cu_objdet_pref = 'http://www.columbia.edu/AIDA/DVMM/Entities/ObjectDetection/'

# RUN00004 .. # May 8, 2019 for D2
#'RUN00005' # May 19, 2019 for D2
merge_version = 'RUN00006' # June 21, 2019 for D3
cu_grndg_ent_pref = 'http://www.columbia.edu/AIDA/DVMM/Entities/GroundingBox/'+merge_version+'/'
cu_grndg_type_pref = 'http://www.columbia.edu/AIDA/DVMM/TypeAssertions/GroundingBox/'+merge_version+'/'
cu_grndg_clstr_img_pref = 'http://www.columbia.edu/AIDA/DVMM/Clusters/BoxOverlap/'+merge_version+'/'
cu_grndg_clstr_txt_pref = 'http://www.columbia.edu/AIDA/DVMM/Clusters/Grounding/'+merge_version+'/'



# USC path always changed 
usc_dict_path = working_path + 'usc_dict/'+ 'uscvision_grounding_output_cu_format_' + p_f + '.pickle' 
add_usc_result = True
if add_usc_result:
    grounding_dict = merge_usc(usc_dict_path, grounding_dict, child_dict)
# Done: the [grounding_dict] will be updated by merged_grounding_dict
usc_pref = 'http://www.usc.edu/AIDA/IRIS/'

#'RUN00002' # May 19, 2019
usc_merge_version = 'RUN00003' # June 22, 2019
usc_grndg_ent_pref = 'http://www.usc.edu/AIDA/IRIS/Entities/GroundingBox/'+usc_merge_version+'/'
usc_grndg_type_pref = 'http://www.usc.edu/AIDA/IRIS/TypeAssertions/GroundingBox/'+usc_merge_version+'/'
usc_grndg_clstr_txt_pref = 'http://www.usc.edu/AIDA/IRIS/Clusters/Grounding/'+merge_version+'/'
print('Check Point: USC gournding path change',usc_dict_path)



rpi_entity_pref = 'http://www.isi.edu/gaia/entities/'
gaia_prefix = 'http://www.isi.edu/gaia'
# nist_ont_pref = '.../SM-KBP/2018/ontologies/InterchangeOntology#'
nist_ont_pref = 'https://tac.nist.gov/tracks/SM-KBP/2019/ontologies/InterchangeOntology#'
justified_by_ = URIRef(nist_ont_pref+'justifiedBy')
entity_ = URIRef(nist_ont_pref+'Entity')
sys_ = URIRef(nist_ont_pref+'system')
hasName_ = URIRef(nist_ont_pref+'hasName')


2019-07-14 02:15:00.046268
Check Point: ISI raw data change /dvmm-filer2/projects/AIDA/data/ldc_eval_m18/LDC2019E42_AIDA_Phase_1_Evaluation_Source_Data_V1.0/
Check Point: version change /home/bobby/aida_copy/AIDA/M18_copy/data/all_features/grounding_dict_PT002_r2.pickle

dict len: 42905
grounding_dict {'http://www.isi.edu/gaia/entities/56ded7a2-9fcf-4875-92bd-0a70d7fa3d5a': {'textual_features': array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), 'name': 'no_name', 'type_rdf': rdflib.term.URIRef('https://tac.nist.gov/tracks/SM-KBP/2019/ontologies/LDCOntology#PER'), 'mentions': {'f725a8fa797484b4383c2e6889ae06f63b393': {'grounding': {'no_image': {}}, 'textual_features': array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), 'name': 'paratroopers', 'sentence': 'The ambush was initiated when DPR forces fired a rocket-propelled grenade at an armoured personnel carrier that had been carrying paratroopers causing the vehicle to explode'}}, 'source_type': 'JPG', 'language': 'English', 'groundin

In [5]:
%%time
###loading visual features
#visual features path
ins_img_path = working_path + 'all_features/instance_features_jpg.lmdb'
ins_kfrm_path = working_path + 'all_features/instance_features_keyframe.lmdb'

sem_img_path = working_path + 'all_features/semantic_features_jpg.lmdb'
sem_kfrm_path = working_path + 'all_features/semantic_features_keyframe.lmdb'

#load instance features
ins_img_env = lmdb.open(ins_img_path, map_size=1e11, readonly=True, lock=False)
ins_img_txn = ins_img_env.begin(write=False)
ins_kfrm_env = lmdb.open(ins_kfrm_path, map_size=1e11, readonly=True, lock=False)
ins_kfrm_txn = ins_kfrm_env.begin(write=False)

#load semantic features
sem_img_env = lmdb.open(sem_img_path, map_size=1e11, readonly=True, lock=False)
sem_img_txn = sem_img_env.begin(write=False)
sem_kfrm_env = lmdb.open(sem_kfrm_path, map_size=1e11, readonly=True, lock=False)
sem_kfrm_txn = sem_kfrm_env.begin(write=False)

CPU times: user 1.2 ms, sys: 469 µs, total: 1.67 ms
Wall time: 7.03 ms


In [6]:
%%time
#undergoing functions
def get_features(key,dtype,ftype):
#     print("get_features for ",ftype)
    if ftype=='instance':
        if dtype=='jpg':
            txn = ins_img_txn
        elif dtype=='keyframe':
            txn = ins_kfrm_txn
        else:
            return []
    elif ftype=='semantic':
        if dtype=='jpg':
            txn = sem_img_txn
        elif dtype=='keyframe':
            txn = sem_kfrm_txn
        else:
            return []
    else:
        return
    
    value = txn.get(key.encode('utf-8'))
    if value!=None:
        return np.frombuffer(value, dtype='float32').tolist()
    return []

CPU times: user 6 µs, sys: 0 ns, total: 6 µs
Wall time: 10.5 µs


# Main Body

In [7]:
%%time 
# for Main Body
import multiprocessing as mp
# maybe changed when different runs
print(datetime.now())
export_dir = working_path + 'merged_ttl/merged_ttl_'+p_f
#graph integration and modification
#entity level

def transferAIF(p_id):
#for k,p_id in enumerate(parent_dict):

    # Todo test
#     if (k<8):
#         continue
#     print('k',k,p_id)
    g = Graph()
    
    #load rpi graph if exists
    if p_id in rpi_ttl_list:
        turtle_path = os.path.join(rpi_ttl_path, p_id+'.ttl')
        turtle_content = open(turtle_path).read()
        g.parse(data=turtle_content, format='n3')
    
    #load and merge cu graph if exists
    if p_id in cu_ttl_list:
        turtle_path = os.path.join(cu_ttl_path, p_id+'.ttl')
        turtle_content = open(turtle_path).read()
        g.parse(data=turtle_content, format='n3')
    
    #load and merge cu graph for instance matching if exists
    if p_id in cu_ttl_ins_list:
        turtle_path = os.path.join(cu_ttl_ins_path, p_id+'.ttl')
        turtle_content = open(turtle_path).read()
        g.parse(data=turtle_content, format='n3')
    
    sys_instance_matching = aifutils.make_system_with_uri(g, cu_pref+'Systems/Instance-Matching/ResNet152')
    sys_grounding = aifutils.make_system_with_uri(g, cu_pref+'Systems/Grounding/ELMo-PNASNET')
    usc_sys_grounding = aifutils.make_system_with_uri(g, usc_pref + 'Systems/ZSGrounder')
    
    #find vision and text entities
    sbj_all = set(g.subjects())
    img_entities = {}
    keyframe_entities = {}
    ltf_entities = {}
    for sbj in sbj_all:
        sbj_name = sbj.toPython()
        if cu_objdet_pref in sbj_name:
            if sbj.__class__ == rdflib.term.URIRef:
                if 'JPG' in sbj_name:
                    img_id = '/'.join(sbj_name.split('/')[-2:])
                    img_entities[img_id] = sbj
                elif 'Keyframe' in sbj_name:
                    kfrm_id = '/'.join(sbj_name.split('/')[-2:])
                    keyframe_entities[kfrm_id] = sbj
        elif rpi_entity_pref in sbj_name:
            if sbj.__class__ == rdflib.term.URIRef and rpi_entity_pref in sbj_name:
                ltf_entities[sbj_name] = sbj
    
    # Done
#     if p_id in []:#['IC0011TIB']:
#         continue
#     print('k',k,p_id)
#     if (g==None):
#         print('p_id', k, p_id)
    
    ##adding private data to entities for cu grounding
    #images
    for key in img_entities:
        dtype='jpg'
        #instance features
        ftype='instance'
        data_instance = get_features(key,dtype,ftype)
        
        #semantic features
        ftype='semantic'
        data_semantic = get_features(key,dtype,ftype)
        
        #aggregation
        j_d_i = json.dumps({'columbia_vector_instance_v1.0': data_instance})
        j_d_s = json.dumps({'columbia_vector_grounding_v1.0': data_semantic})
        entity = img_entities[key]
        aifutils.mark_private_data(g, entity, j_d_i, sys_instance_matching)
        aifutils.mark_private_data(g, entity, j_d_s, sys_grounding)
    
    #keyframes  
    for key in keyframe_entities:
        dtype='keyframe'
        #instance features
        ftype='instance'
        data_instance = get_features(key,dtype,ftype)

        #semantic features
        ftype='semantic'
        data_semantic = get_features(key,dtype,ftype)

        #aggregation
        j_d_i = json.dumps({'columbia_vector_instance_v1.0': data_instance})
        j_d_s = json.dumps({'columbia_vector_grounding_v1.0': data_semantic})
        entity = keyframe_entities[key]
        aifutils.mark_private_data(g, entity, j_d_i, sys_instance_matching)
        aifutils.mark_private_data(g, entity, j_d_s, sys_grounding)
        
        

    cnt_img = {}
    cnt_boxO = {}
    cnt_ltf = {}
    #add text features, grounding, linking
    for key in ltf_entities:        
        if key not in grounding_dict:
            continue
        entity_name = None
        USC_GROUNDING = 'usc_vision' in grounding_dict[key]['grounding'].values()
        if not USC_GROUNDING:
#             print('our grounding')
            #text features
            j_d_t = json.dumps({'columbia_vector_text_v1.0': grounding_dict[key]['textual_features'].tolist()})
            entity_ltf = ltf_entities[key]
            aifutils.mark_private_data(g, entity_ltf, j_d_t, sys_grounding)
        
            #type and name of entity to be linked
            type_rdf = grounding_dict[key]['type_rdf']
            entity_name = grounding_dict[key]['name']
            grndg_file_type = grounding_dict[key]['source_type']
            
        if entity_name is None:
            continue

        
        #keep track of entities with same names for avoiding clustering overlap
        if entity_name in cnt_ltf:
            cnt_ltf[entity_name] += 1
        else:
            cnt_ltf[entity_name] = 1
        
        clstr_prot_flag = False #cluster obj for entity_ltf not created yet
        #adding grounding bboxes as new entities
        for img_id in grounding_dict[key]['grounding']:
            if img_id == 'system':
                continue
            grnd = grounding_dict[key]['grounding'][img_id]
            for ii,bbox in enumerate(grnd['bbox']):
                
                if img_id in cnt_img: #to keep track of cnt of bbox of same image
                    cnt_img[img_id] += 1
                else:
                    cnt_img[img_id] = 1
                #add grounding bbox as entity
                score = grnd['bbox_score'][ii]
                if not USC_GROUNDING:
                    type_eid = cu_grndg_type_pref+f"{grndg_file_type}/{img_id.split('.')[0]}/{cnt_img[img_id]}/ERE"
                    ent_eid = cu_grndg_ent_pref+f"{grndg_file_type}/{img_id.split('.')[0]}/{cnt_img[img_id]}"
                    entity_grnd = aifutils.make_entity(g, ent_eid, sys_grounding)
                    type_assertion = aifutils.mark_type(g, type_eid, entity_grnd, type_rdf, sys_grounding, score)
                elif USC_GROUNDING:
                    type_eid = usc_grndg_type_pref+f"{grndg_file_type}/{img_id.split('.')[0]}/{cnt_img[img_id]}/ERE"
                    ent_eid = usc_grndg_ent_pref+f"{grndg_file_type}/{img_id.split('.')[0]}/{cnt_img[img_id]}"
                    entity_grnd = aifutils.make_entity(g, ent_eid, usc_sys_grounding)
                    type_assertion = aifutils.mark_type(g, type_eid, entity_grnd, type_rdf, usc_sys_grounding, score)
              
                
                # Done: 
                # 1. add if for the branches for image and keyframe.
                # 2. add aifutils.mark_keyframe_video_justification
                # 3. check output 
                # aifutils.mark_keyframe_video_justification(g, [entity, type_assertion], "NYT_ENG_20181231_03", "keyframe ID",
                #                                                    bb2, system, 0.234)
                # source: HC0005BR6_23
                # print(img_id)

                # Done: 
                # merge usc_grounding dict
                # add usc_grounding entities and clusters
                
                # Test 
#                 print("type_assertion",type_assertion, img_id)

                
                bb = Bounding_Box((bbox[0], bbox[1]), (bbox[2], bbox[3]))
                if not USC_GROUNDING:
                    if 'JPG' in type_assertion:
                        imgid = img_id.split('.')[0]
                        justif = aifutils.mark_image_justification(g, [entity_grnd, type_assertion], imgid, bb, sys_grounding, score)
#                     
                    elif 'Keyframe' in type_assertion:
                        imgid = img_id.split('.')[0].split('_')[0]
                        kfid = img_id.split('.')[0].split('_')[1] # it should be keyframe image id or keyframe number
                        justif = aifutils.mark_keyframe_video_justification(g, [entity, type_assertion], imgid, kfid, \
                                                                       bb, sys_grounding, score)
                elif USC_GROUNDING:
                    imgid = img_id.split('.')[0]
                    justif = aifutils.mark_image_justification(g, [entity_grnd, type_assertion], imgid, bb, usc_sys_grounding, score)
                else:
                    print('[Merge Error] in Main Body: the type_assertion is wrong')
                aifutils.add_source_document_to_justification(g, justif, p_id)
                aifutils.mark_informative_justification(g, entity_grnd, justif)
                
            
                if not USC_GROUNDING:
                    grounding_features = grnd['grounding_features'][ii].tolist()
                    instance_features = grnd['instance_features'][ii].tolist()
                    #add private data to this very bbox entity
                    j_d_g = json.dumps({'columbia_vector_grounding_v1.0': grounding_features})
                    j_d_i = json.dumps({'columbia_vector_instance_v1.0': instance_features})
                    aifutils.mark_private_data(g, entity_grnd, j_d_g, sys_grounding)
                    aifutils.mark_private_data(g, entity_grnd, j_d_i, sys_instance_matching)

                
                #### add clusters
                # Grounding Cluster
                # Done: filtering about punctuation
#                 translator = str.maketrans(string.punctuation, '_'*len(string.punctuation),'' )
#                 'entity_name'.translate(translator)
                entity_name_tmp = '_'.join(entity_name.split(' '))
                entity_name_in_IRI = "".join(x if x.isalpha() or x.isdigit() or x =='_' else '-' for x in entity_name_tmp)
                # '_'.join(entity_name.split(' '))
                #gbbox entity to rpi entity
                if not USC_GROUNDING:
                    if not clstr_prot_flag: #create cluster if not present
                        clst_eid = cu_grndg_clstr_txt_pref+f"{entity_name_in_IRI}/{cnt_ltf[entity_name]}"
                        clusterObj = aifutils.make_cluster_with_prototype(g, clst_eid, entity_ltf, sys_grounding)
                        clstr_prot_flag = True
                    #cluster current bbox with current ltf_entity
                    score = grnd['men-img-score'][ii]
                    aifutils.mark_as_possible_cluster_member(g, entity_grnd, clusterObj, score, sys_grounding)
                    # Done: add prototype as member
                    aifutils.mark_as_possible_cluster_member(g, entity_ltf, clusterObj, 1, sys_grounding)
                elif USC_GROUNDING:
                    if not clstr_prot_flag: #create cluster if not present
                        clst_eid = usc_grndg_clstr_txt_pref+f"{entity_name_in_IRI}/{cnt_ltf[entity_name]}"
                        clusterObj = aifutils.make_cluster_with_prototype(g, clst_eid, entity_ltf, usc_sys_grounding)
                        clstr_prot_flag = True
                    #cluster current bbox with current ltf_entity
                    score = grnd['men-img-score'][ii]
                    aifutils.mark_as_possible_cluster_member(g, entity_grnd, clusterObj, score, usc_sys_grounding)
                    # Done: add prototype as member
                    aifutils.mark_as_possible_cluster_member(g, entity_ltf, clusterObj, 1, usc_sys_grounding)
                    
                # BoundingBox Overlap Cluster (Instance Matching)
                #gbbox entity to objdet entity for instance matching
                if not USC_GROUNDING:
                    clstr_prot_b2b_flag = False
                    for jj,img_id_link in enumerate(grnd['link_ids'][ii]): #for all objdet bboxes
                        if img_id_link in img_entities:
                            entity_link_img = img_entities[img_id_link]
                        elif img_id_link in keyframe_entities:
                            entity_link_img = keyframe_entities[img_id_link]
                        else:
                            continue
                        if img_id in cnt_boxO: #to keep track of cnt of bbox overlap for same image
                            cnt_boxO[img_id] += 1
                        else:
                            cnt_boxO[img_id] = 1
                        if not clstr_prot_b2b_flag:
                            clst_b2b_eid = cu_grndg_clstr_img_pref+f"{img_id.split('.')[0]}/{cnt_boxO[img_id]}"
                            clusterObj_b2b = aifutils.make_cluster_with_prototype(g, clst_b2b_eid, entity_grnd, sys_grounding) # sys_instance_matching
                            clstr_prot_b2b_flag = True

                        score = grnd['link_scores'][ii][jj] #IoU of grnd bbox and objdet bbox
                        aifutils.mark_as_possible_cluster_member(g, entity_link_img, clusterObj_b2b, score, sys_grounding) # sys_instance_matching
                        # Done: add prototype as member
                        aifutils.mark_as_possible_cluster_member(g, entity_grnd, clusterObj_b2b, 1, sys_grounding) # sys_instance_matching
                    
    # Check Point: merged_ttl_D2
#     /data/bobby/AIDA/M18_copy/data/merged_ttl/merged_ttl_D2/
#     IC0011VEA.ttl
#     GroundingBox
    with open(os.path.join(export_dir, p_id+'.ttl'), 'w') as fout:
        serialization = BytesIO()
        g.serialize(destination=serialization, format='turtle')
        fout.write(serialization.getvalue().decode('utf-8'))
    #sys.stdout.write('Key {}/{} \r'.format(k,len(parent_dict)))                
    sys.stdout.flush()
pool = mp.Pool(processes=32)
    #for x,y in candidateDic.items():
    #print candidateDic.keys()
res = pool.map(transferAIF, parent_dict.keys())

print(datetime.now())

2019-07-14 02:15:19.672818
2019-07-14 02:37:37.532964
CPU times: user 2.23 s, sys: 2.64 s, total: 4.87 s
Wall time: 22min 17s


# DBPedia to Link

In [ ]:
def get_DB2link(graph):
    '''
    A function that gets graph and loads information in it.
    '''
    #get data and put in entity2mention dictionary
    DB2en = {}
    
    entities = graph.subjects(predicate=RDF.type,object=entity_)
    for entity in entities:
        entity_id = entity.toPython()
        link_nodes = list(g.objects(subject=entity,
                                   predicate=URIRef(nist_ont_pref+'link')))
        if len(link_nodes)==0:
            continue
        link_node = link_nodes[0]
        link_string = list(g.objects(subject=link_node,
                       predicate=URIRef(nist_ont_pref+'linkTarget')))[0].toPython()
        
        if link_string not in DB2en:
            DB2en[link_string] = [entity]
        else:
            DB2en[link_string].append(entity)
    return DB2en

In [ ]:
%%time 
#10 mins
path_pref = '/home/bobby/aida_copy/AIDA/M18_copy/data/'

RPI_AIF_path = rpi_ttl_path #path_pref + 'rpi_ttl/RPI_TA1_20190504'
#'rpi_ttl/RPI_TA1_aif_dryrun20190410'#'rpi_ttl/RPI_TA1a_02102019_rerun_R4' #'rpi_ttl/RPI_dryrun_D1'
# nist_ont_pref = '.../SM-KBP/2018/ontologies/InterchangeOntology#'
nist_ont_pref = 'https://tac.nist.gov/tracks/SM-KBP/2019/ontologies/InterchangeOntology#'
entity_ = URIRef(nist_ont_pref+'Entity')

print('Creating id2link dictionary...')
id2link = {}
turtle_files = os.listdir(RPI_AIF_path)
for i,file in enumerate(turtle_files):
    if ".ttl" not in file:
        continue
    turtle_path = os.path.join(RPI_AIF_path, file)
    #loading turtle content
    turtle_content = open(turtle_path).read()
    g = Graph().parse(data=turtle_content, format='n3')
    id_ = file.split('.')[0]
    id2link[id_] = get_DB2link(g)
    
    sys.stdout.write('File {}/{} \r'.format(i+1,len(turtle_files)))                
    sys.stdout.flush()

# Todo: open to write
with open(path_pref + 'tmp/id2link_dict_'+p_f+'.pickle', 'wb') as f:
    pickle.dump(id2link,f,protocol=pickle.HIGHEST_PROTOCOL)



In [ ]:
# Check Point: id2link printing
print(top_dict(id2link))

In [ ]:
# def add_private_data(g, img_entities, dtype):
# #     print('add_private_data for instance and semantic:', dtype)
#     for key in img_entities:
#         #instance features
#         ftype='instance'
#         data_instance = get_features(key,dtype,ftype)
        
#         #semantic features
#         ftype='semantic'
#         data_semantic = get_features(key,dtype,ftype)
        
#         #aggregation
#         j_d_i = json.dumps({'columbia_vector_instance_v1.0': data_instance})
#         j_d_s = json.dumps({'columbia_vector_grounding_v1.0': data_semantic})
#         entity = img_entities[key]
#         aifutils.mark_private_data(g, entity, j_d_i, sys_instance_matching)
#         aifutils.mark_private_data(g, entity, j_d_s, sys_grounding)

#         return g

    ##adding private data to entities
    #images
#     for key in img_entities:
#         dtype='jpg'
#         #instance features
#         ftype='instance'
#         data_instance = get_features(key,dtype,ftype)
        
#         #semantic features
#         ftype='semantic'
#         data_semantic = get_features(key,dtype,ftype)
        
#         #aggregation
#         j_d_i = json.dumps({'columbia_vector_instance_v1.0': data_instance})
#         j_d_s = json.dumps({'columbia_vector_grounding_v1.0': data_semantic})
#         entity = img_entities[key]
#         aifutils.mark_private_data(g, entity, j_d_i, sys_instance_matching)
#         aifutils.mark_private_data(g, entity, j_d_s, sys_grounding)
    
    #keyframes  
#     for key in keyframe_entities:
#         dtype='keyframe'
#         #instance features
#         ftype='instance'
#         data_instance = get_features(key,dtype,ftype)

#         #semantic features
#         ftype='semantic'
#         data_semantic = get_features(key,dtype,ftype)

#         #aggregation
#         j_d_i = json.dumps({'columbia_vector_instance_v1.0': data_instance})
#         j_d_s = json.dumps({'columbia_vector_grounding_v1.0': data_semantic})
#         entity = keyframe_entities[key]
#         aifutils.mark_private_data(g, entity, j_d_i, sys_instance_matching)
#         aifutils.mark_private_data(g, entity, j_d_s, sys_grounding)


# Testing

In [ ]:
%%time
rpi_en_path = path_pref + 'rpi_ttl/ttl_en_r0'
rpi_en_list = set([f.split('.')[0] for f in os.listdir(rpi_en_path)])
for k,p_id in enumerate(rpi_ttl_list):
    g = Graph()
    
    #load rpi graph if exists
    if p_id in rpi_en_list:
        turtle_path = os.path.join(rpi_en_path, p_id+'.turtle')
        turtle_content = open(turtle_path).read()
        g.parse(data=turtle_content, format='n3')
        entities = list(g.subjects(predicate=RDF.type,object=entity_))
        for entity in entities:
            if entity.toPython() in grounding_dict:
                if len(grounding_dict[entity.toPython()]['grounding'])>0:
                    for img_id in grounding_dict[entity.toPython()]['grounding']:
                        if len(grounding_dict[entity.toPython()]['grounding'][img_id]['link_ids'])>0:
                            print(p_id)
    sys.stdout.write('Key {}/{} \r'.format(k,len(rpi_en_list)))                
    sys.stdout.flush()

In [ ]:
grounding_dict[entity.toPython()]['grounding']

In [ ]:
with open(path_pref + 'tmp/entity2mention_dict_r0.pickle', 'rb') as f:
    en2men = pickle.load(f)

with open(path_pref + 'tmp/id2mentions_dict_r0.pickle', 'rb') as f:
    id2men = pickle.load(f)

In [ ]:
test_ttl = '/home/hassan/local/M9/rpi_ttl/ttl_en_r0/HC0000CVV.turtle'
g = Graph()
ttl_content = open(test_ttl).read()
g.parse(data=ttl_content, format='n3')
#json_string = g.serialize(format='json-ld', indent=4)
#json_dict = json.loads(json_string)
entities = list(g.subjects(predicate=RDF.type,object=entity_))

In [ ]:
entity = URIRef('http://www.isi.edu/gaia/entities/00668720-706b-415c-b76e-4b5419ef77ea')
hasName_ = URIRef(nist_ont_pref+'hasName')
entity_hasName = list(g.objects(predicate=hasName_,subject=entity))

In [ ]:
e = np.array([])

In [ ]:
sen_batch[1]

In [ ]:
entity2mention = utils.get_entity2mention(g)

In [ ]:
en2men['http://www.isi.edu/gaia/entities/00668720-706b-415c-b76e-4b5419ef77ea']

In [ ]:
grounding_dict['http://www.isi.edu/gaia/entities/00668720-706b-415c-b76e-4b5419ef77ea']

In [ ]:
link_ = URIRef(nist_ont_pref+'link')
link_target_ = URIRef(nist_ont_pref+'linkTarget')
for entity in entities:
list(g.triples((None,link_,None)))

In [ ]:
for just in just_by:
    pv_data_rdf = list(g.objects(subject=just,
                            predicate=URIRef(nist_ont_pref+'privateData')))[0]
    dict_str=list(g.objects(subject=pv_data_rdf,
                            predicate=URIRef(nist_ont_pref+'jsonContent')))[0].toPython()
    print(dict_str)

In [ ]:
path_dict = utils.create_path_dict(os.path.join(corpus_path,'data/ltf/ltf/'))
en2men = utils.get_entity2mention(g)
filter_out=['pronominal_mention','GeopoliticalEntity','Organization','Location']
id2men = utils.create_entity_dict(en2men, path_dict, caption_alignment_path=[], filter_out=filter_out)

In [ ]:
id2men

In [ ]:
list(g.triples((None,None,None)))